## Higgs Effective Field Theory (HEFT) Study: Prepare Training Data
> Created: Feb 15, 2024 Nicola de Filippis, Kurtis Johnson, Harrison B. Prosper<br>

### Introduction

In this project, the physics model of interest is the Higgs effective field theory (HEFT), defined by a 5D parameter space of Wilson coefficients [1]:
\begin{align}
    \theta & = c_{hhh}, c_t, c_{tt}, c_{ggh}, c_{gghh}.
\end{align}
For this investigation, we restrict our attention to a single observable, namely, the di-Higgs mass denoted by $m_{hh}$ and, in our proof-of-principle, we set $c_{hhh} = c_t = 1$. The Standard Model (SM) values for these parameters are $c_{hhh} = c_t = 1$, and $c_{tt} = c_{ggh} = c_{gghh} = 0$. In general, we would consider multiple observables simultaneously.

We shall use simulation-based inference to construct **confidence sets**, at confidence level (CL) $\tau$, in the HEFT and later SMEFT parameter spaces. This requires approximating $\mathbb{P}(\lambda \le \lambda_0 | \theta) = \mathbb{E}(Z | \theta)$ [2,3], which we shall do in two different ways: 1) using a multi-dimensional histogram constructed using a kdtree and 2) a machine learning (ML) model, where for a given hypothesis $H_0: \theta = \theta_0$ versus $H_1: \theta \ne \theta_0$, $\lambda_0$ is the observed value of a test statistic $\lambda$ with the property that large values of the test statistic *disfavor* the hypothesis $H_0$. 

Given the cumulative distribution function (cdf), $\mathbb{P}(\lambda \le \lambda_0 | \theta)$, a confidence set at CL $\tau$ is the set of $\theta$ values for which $\mathbb{P}(\lambda \le \lambda_0 | \theta) \leq \tau$.  By definition, a confidence set (in the simplest case in which we restrict ourselves to a single class of experiments) is an observation-dependent set of $\theta$ values which upon repeated replication of the experiment is guaranteed to include the true value of $\theta$ a fraction $\ge \tau$.


### Training data
In principle [2,3], to approximate $\mathbb{P}(\lambda \le \lambda_0 | \theta)$ we need to sample the HEFT parameter space sufficiently densely and, at every point, simulate data like the ones actually observed. However, if it is computationally infeasible to sample the parameter space densely enough, the next option is to use a sparser sampling of the parameter space and reweight existing simulated data to mimic the sampling of data at any other point $\theta$. To do this requires knowledge of the cross section per binned observables, or the differential cross section,  as a function of both the *observables* and the *parameters*; here, $m_{hh}$ and $\theta$, respectively. Therefore, one sub-goal of the project is construct a parametrization of this function.

This notebook prepares the data needed to train a model of the di-Higgs cross section as a function of $m_{hh}$ and the Wilson coefficients, $\theta$.

### References
  1. Lina Alasfar *et al.*, arXiv:2304.01968v1
  2. Ann Lee *et al.*, https://arxiv.org/abs/2107.03920
  3. Ali Al Kadhim *et al.*, https://iopscience.iop.org/article/10.1088/2632-2153/ad218e


In [1]:
import os, sys

# the standard module for tabular data
import pandas as pd

# the standard module for array manipulation
import numpy as np

# the standard modules for high-quality plots
import matplotlib as mp
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# update fonts
FONTSIZE = 16
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : FONTSIZE}
mp.rc('font', **font)

# set usetex = False if LaTex is not 
# available on your system or if the 
# rendering is too slow
mp.rc('text', usetex=True)

### Load $\texttt{POWHEG}$ data

Note 1: $\kappa_\lambda \equiv c_{hhh}$.

Note 2: The bin widths in $m_{hh}$ is 15 GeV.

In [3]:
def read_data(datafiles):
    df = []
    for datafile in datafiles:
        print('reading %s' % datafile)
        df.append( pd.read_csv(datafile) )

    # concatenate dataframes
    df = pd.concat(df)
    
    # select points with klambda=CT=1
    select = (df.klambda==1) * (df.CT==1)
    
    # make number of rows be a multiple of 20
    total  = select.sum()
    total  = int(total / 20)
    total  = 20 * total
    
    df = df[select][:total]
    print('\nnumber of rows read: %d\n' % len(df))

    # randomly shuffle order of rows in dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    return df

In [4]:
datafiles = ['../data/powheg_total_param_closeBP.csv', 
             '../data/powheg_total_param_closeBP_all.csv']

dfBP = read_data(datafiles)
dfBP[:5]

reading ../data/powheg_total_param_closeBP.csv
reading ../data/powheg_total_param_closeBP_all.csv

number of rows read: 340



,klambda,CT,CTT,CGHH,CGGHH,0,1,2,3,4,...,92,93,94,95,96,97,98,99,100,101
0,1.0,1.0,-3.0,0.8000,0.6000,0.0,0.0,0.0,0.0,0.0,...,0.014107,0.009954,0.016596,0.008322,0.005812,0.006633,0.009131,0.006645,0.010793,0.165184
1,1.0,1.0,-3.0,-0.5333,0.0666,0.0,0.0,0.0,0.0,0.0,...,0.005675,0.005679,0.011560,0.007955,0.005687,0.004543,0.006809,0.000000,0.004551,0.085272
2,1.0,1.0,-1.5,-0.5333,0.0666,0.0,0.0,0.0,0.0,0.0,...,0.003054,0.002093,0.001017,0.002714,0.002037,0.000678,0.001696,0.002378,0.000681,0.024444
3,1.0,1.0,-3.0,-0.8000,1.0000,0.0,0.0,0.0,0.0,0.0,...,0.009819,0.015541,0.018812,0.009818,0.016354,0.008987,0.010645,0.008175,0.013896,0.297774
4,1.0,1.0,0.5,0.5333,0.2666,0.0,0.0,0.0,0.0,0.0,...,0.000905,0.000590,0.000590,0.000275,0.000670,0.000315,0.000275,0.000197,0.000552,0.010214


### Convert to $\texttt{numpy}$ arrays
 1. Get Wilson coefficients and cross sections / bin
 2. Protect against negative cross sections
 3. Pick cross sections from bin 17 (first non-zero bin) to 96

In [5]:
def get_column_names(df, first='17', last='96'):
    # get column names
    columns = list(df.columns)
    params  = columns[2:5]
    
    firstbin= columns.index(first)
    lastbin = columns.index(last)
    
    bins = columns[firstbin:lastbin+1]
    return params, bins

params, bins = get_column_names(dfBP)

params, bins[0], bins[-1], len(bins)

(['CTT', 'CGHH', 'CGGHH'], '17', '96', 80)

In [6]:
# get Wilson coefficients
wilson = dfBP[params].to_numpy()
print(wilson.shape)

# get cross section data
BP   = dfBP[bins].to_numpy()
print(BP.shape)

# protect against negative cross sections
BP   = np.where(BP < 0, 0, BP) 

print(f'cross section/bin (min, mean, max, std): '\
      f'{BP.min():8.4f} pb, {BP.mean():8.4f} pb, {BP.max():8.4f} pb, {BP.std():8.4f} pb')

BP[0]

(340, 3)
(340, 80)
cross section/bin (min, mean, max, std):   0.0000 pb,   0.1135 pb,   2.3421 pb,   0.2548 pb


array([0.02407731, 0.17510256, 0.23824118, 0.32042781, 0.39174527,
       0.45912027, 0.57024157, 0.75990307, 0.90820122, 1.00061119,
       0.94597059, 0.969796  , 0.96058303, 0.90103775, 0.8180384 ,
       0.76312649, 0.74367326, 0.65456909, 0.63892704, 0.55932951,
       0.53316087, 0.44809139, 0.45139596, 0.43554181, 0.39522541,
       0.32176715, 0.34317371, 0.28011903, 0.2708886 , 0.24680381,
       0.25010866, 0.21572046, 0.17761239, 0.16763133, 0.17011237,
       0.14853491, 0.16018283, 0.11207116, 0.11788595, 0.11288231,
       0.11539257, 0.09380305, 0.08212689, 0.08048651, 0.0888485 ,
       0.06307095, 0.07219759, 0.05974938, 0.05311216, 0.06226195,
       0.05145086, 0.04564272, 0.04065108, 0.03817984, 0.03486554,
       0.02489841, 0.046458  , 0.03484802, 0.02323306, 0.01659981,
       0.0298755 , 0.02239429, 0.03816197, 0.02239507, 0.02240941,
       0.02164077, 0.02074356, 0.01494333, 0.01743108, 0.01243484,
       0.01991002, 0.01244469, 0.01494121, 0.00995445, 0.01245

### Save shuffled spectra

In [7]:
# first place data in a dictionary
dmap = {'CTT': wilson.T[0], 
        'CGGH': wilson.T[1], 
        'CGGHH': wilson.T[2]}

dmap.update( {key: BP.T[i] for i, key in enumerate(bins)} )

# then save to a csv file
df = pd.DataFrame(dmap)
df.to_csv('../data/heft_spectra.csv', index=False)

len(df), df[:5]

(340,
    CTT    CGGH   CGGHH        17        18        19        20        21  \
 0 -3.0  0.8000  0.6000  0.024077  0.175103  0.238241  0.320428  0.391745   
 1 -3.0 -0.5333  0.0666  0.052271  0.410202  0.586312  0.636484  0.675402   
 2 -1.5 -0.5333  0.0666  0.016629  0.110292  0.163538  0.191439  0.212770   
 3 -3.0 -0.8000  1.0000  0.034356  0.251041  0.308580  0.359859  0.426314   
 4  0.5  0.5333  0.2666  0.005634  0.039345  0.042716  0.039986  0.036970   
 
          22        23  ...        87        88        89        90        91  \
 0  0.459120  0.570242  ...  0.019910  0.012445  0.014941  0.009954  0.012454   
 1  0.754226  0.884093  ...  0.004547  0.005024  0.013643  0.010234  0.011364   
 2  0.230752  0.277579  ...  0.003058  0.002040  0.003398  0.002038  0.002034   
 3  0.478569  0.572455  ...  0.031875  0.018003  0.014714  0.021289  0.018812   
 4  0.034295  0.029465  ...  0.000713  0.000709  0.000632  0.000826  0.000354   
 
          92        93        94        95

### Save training data to a csv file

Format: $c_{tt}$, $c_{ggh}$, $c_{ggh}$, $m_{hh}$, $\sigma$

Save data from the last 300 spectra and keep the first 40 spectra to test the spectrum model.

In [12]:
# compute mid-points of di-Higgs mass bins, with the mass range mapped to the
# unit interval
_, xbins = BP.shape
xmin = 0
xmax = xbins/100 # bin width is 0.01, which corresponds to 15 GeV

x = np.linspace(xmin, xmax, xbins+1)
# x[1:] = x[1], x[2] ...,x[n-1]
# x[:-1]= x[0], x[1],...,x[n-2]
x = (x[1:]+x[:-1])/2

# fill list with training data 
START = 40
data  = []
for params, spectrum in zip(wilson[START:], BP[START:]):
    p = list(params)

    # increase the incidence of spectra with 
    # cross sections < 0.5 pb
    total_xsec = spectrum.sum()
    if total_xsec < 0.5:
        N = 1
    else:
        N = 1
        
    d = []
    for _ in range(N):
        for mhh, sigma in zip(x, spectrum):
            d = p + [mhh, sigma] # CTT, CGGH, CGGHH, MHH, SIGMA
            data.append(d)

# randomly shuffle rows
data = np.array(data)
np.random.shuffle(data)

N = len(data) // 1000
N *= 1000
data = data[:N]
print(f'training data size: {len(data):5d}')

# create dataframe
data = data.T  # transpose to shape (5, N)
df = pd.DataFrame({'CTT':   data[0], 
                   'CGGH':  data[1], 
                   'CGGHH': data[2], 
                   'mhh':   data[3], 
                   'sigma': data[4]})

# save training data to a csv file
df.to_csv('../data/heft_traindata.csv', index=False)

df

training data size: 24000


,CTT,CGGH,CGGHH,mhh,sigma
0,-1.5,-0.4000,0.2666,0.015,0.077345
1,0.5,1.0000,0.0000,0.385,0.002021
2,0.0,0.5333,0.0666,0.565,0.000226
3,-1.0,-0.4000,0.0000,0.445,0.010373
4,-1.0,0.5333,0.3333,0.565,0.004299
...,...,...,...,...,...
23995,-3.0,0.1333,0.3333,0.055,0.544932
23996,-1.5,-1.0000,1.0000,0.455,0.032078
23997,0.5,-0.5333,0.0000,0.295,0.002514
23998,-1.5,-0.8000,0.6000,0.595,0.009938
